# Setup

In [1]:
using JuMP
using Gurobi
using CSV
using DataFrames

Precompiling JuMP
  ✓ Compat
  ✓ TranscodingStreams
  ✓ Compat → CompatLinearAlgebraExt
  ✓ TranscodingStreams → TestExt
  ✓ CodecZlib
  ✓ CodecBzip2
  ✓ ChainRulesCore
  ✓ DataStructures
  ✓ ChainRulesCore → ChainRulesCoreSparseArraysExt
  ✓ LogExpFunctions → LogExpFunctionsChainRulesCoreExt
  ✓ MutableArithmetics
  ✓ SpecialFunctions → SpecialFunctionsChainRulesCoreExt
  ✓ StaticArrays
  ✓ StaticArrays → StaticArraysStatisticsExt
  ✓ ForwardDiff → ForwardDiffStaticArraysExt
  ✓ MathOptInterface
  ✓ JuMP
  17 dependencies successfully precompiled in 53 seconds. 28 already precompiled.
Precompiling Gurobi
  ✓ Gurobi
  1 dependency successfully precompiled in 2 seconds. 45 already precompiled.
Precompiling CSV
  ✓ InlineStrings
  ✓ Tables
  ✓ SentinelArrays
  ✓ WeakRefStrings
  ✓ CSV
  5 dependencies successfully precompiled in 8 seconds. 20 already precompiled.
Precompiling DataFrames
  ✓ SortingAlgorithms
  ✓ PrettyTables
  ✓ DataFrames
  3 dependencies successfully precompiled in 26 

In [2]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-18


### Data importation

In [4]:
centers = CSV.File("data/centers.csv",header=0) |> Tables.matrix;
stations = CSV.File("data/stations.csv",header=0) |> Tables.matrix;
landfills = CSV.File("data/landfills.csv",header=0) |> Tables.matrix;
q = CSV.File("data/q.csv",header=0) |> Tables.matrix;

centers2 = CSV.File("data/centers2.csv",header=0) |> Tables.matrix;
stations2 = CSV.File("data/stations2.csv",header=0) |> Tables.matrix;
landfills2 = CSV.File("data/landfills2.csv",header=0) |> Tables.matrix;
q2 = CSV.File("data/q2.csv",header=0) |> Tables.matrix;

centers_all = [centers;centers2];
stations_all = [stations;stations2];
landfills_all = [landfills;landfills2];
q_all = [q;q2];

n1 = size(centers)[1]
s1 = size(stations)[1]
m1 = size(landfills)[1]
n2 = size(centers2)[1]
s2 = size(stations2)[1]
m2 = size(landfills2)[1]
n_all = n1+n2;
s_all = s1+s2;
m_all = m1+m2;

# Part A

## Calculate all distances

In [5]:
function euclidean_distance(x1, y1, x2, y2)
    return sqrt((x1 - x2)^2 + (y1 - y2)^2)
end

dist = zeros(50, 15)
for i in 1:50
    for j in 1:15
        dist[i, j] = euclidean_distance(centers[i, 1], centers[i, 2], landfills[j, 1], landfills[j, 2])
    end
end


## Create Model

In [6]:
model = Model(Gurobi.Optimizer)

# Binary decision variables
@variable(model, y[1:15], Bin)  # y_j: whether landfill j is built
@variable(model, x[1:50, 1:15], Bin)  # x_ij: whether center i sends waste to landfill j

# Objective: Minimize transportation cost
@objective(model, Min, sum(q[i] * dist[i, j] * x[i, j] for i in 1:50, j in 1:15))

# Constraint: Each center sends waste to exactly one landfill
@constraint(model, [i in 1:50], sum(x[i, j] for j in 1:15) == 1)

# Constraint: Only 5 landfills can be built
@constraint(model, sum(y[j] for j in 1:15) == 5)

# Constraint: Waste can only go to a landfill if it's built
@constraint(model, [i in 1:50, j in 1:15], x[i, j] <= y[j])


Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-18


50×15 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 -y[1] + x[1,1] ≤ 0   -y[2] + x[1,2] ≤ 0   …  -y[15] + x[1,15] ≤ 0
 -y[1] + x[2,1] ≤ 0   -y[2] + x[2,2] ≤ 0      -y[15] + x[2,15] ≤ 0
 -y[1] + x[3,1] ≤ 0   -y[2] + x[3,2] ≤ 0      -y[15] + x[3,15] ≤ 0
 -y[1] + x[4,1] ≤ 0   -y[2] + x[4,2] ≤ 0      -y[15] + x[4,15] ≤ 0
 -y[1] + x[5,1] ≤ 0   -y[2] + x[5,2] ≤ 0      -y[15] + x[5,15] ≤ 0
 -y[1] + x[6,1] ≤ 0   -y[2] + x[6,2] ≤ 0   …  -y[15] + x[6,15] ≤ 0
 -y[1] + x[7,1] ≤ 0   -y[2] + x[7,2] ≤ 0      -y[15] + x[7,15] ≤ 0
 -y[1] + x[8,1] ≤ 0   -y[2] + x[8,2] ≤ 0      -y[15] + x[8,15] ≤ 0
 -y[1] + x[9,1] ≤ 0   -y[2] + x[9,2] ≤ 0      -y[15] + x[9,15] ≤ 0
 -y[1] + x[10,1] ≤ 0  -y[2] + x[10,2] ≤ 0     -y[15] + x[10,15] ≤ 0
 -y[1] + x[11,1] ≤ 0  -y[2] + x[11,2] ≤ 0  …  -y[15] + x[11,15] ≤ 0
 -y[1] + x[12,1] ≤ 0  -y[2] + x[12,2] ≤ 0     -y[15] + x[12,15] ≤ 0
 -y[1] + x[13,1] ≤ 0  -y[2

## Run model

In [7]:
optimize!(model)


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 801 rows, 765 columns and 2265 nonzeros
Model fingerprint: 0xc96c1769
Variable types: 0 continuous, 765 integer (765 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+03, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve time: 0.00s
Presolved: 801 rows, 765 columns, 2265 nonzeros
Variable types: 0 continuous, 765 integer (765 binary)
Found heuristic solution: objective 1026891.5990

Root relaxation: objective 8.404875e+05, 168 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    840487.52400 840487.524  0.00%     -    0s

Explored 1 nodes (168

In [8]:
# Get the built landfills
built_landfills = findall(y -> value(y) > 0.5, y)  # These are the landfills that are built

# Calculate the total distance traveled in miles-tons
total_distance = objective_value(model)

println("Built landfills: ", built_landfills)
println("Total distance (miles-tons): ", total_distance)


Built landfills: [3, 4, 7, 13, 14]
Total distance (miles-tons): 840487.5240012797


# Part B